In [1]:
system("pip3 show dxpy", intern = TRUE)

[1] "Name: dxpy"                                                                                                                            
 [2] "Version: 0.377.0"                                                                                                                      
 [3] "Summary: DNAnexus Platform API bindings for Python"                                                                                    
 [4] "Home-page: https://github.com/dnanexus/dx-toolkit"                                                                                     
 [5] "Author: Aleksandra Zalcman, Andrey Kislyuk, Anurag Biyani, Geet Duggal, Katherine Lai, Kurt Jensen, Marek Hrvol, Ohad Rodeh, Phil Sung"
 [6] "Author-email: support@dnanexus.com"                                                                                                    
 [7] "License: Apache Software License"                                                                                                      
 [8] "Location: /opt/conda/lib/python3.11/site-packages"                                                                                     
 [9] "Requires: argcomplete, certifi, psutil, python-dateutil, urllib3, websocket-client"                                                    
[10] "Required-by: "

In [2]:
library(glue)
library(data.table)

In [3]:
projectid <- "project-GvZyZ20J81vgPJGbJy8pgpyq"
rid <- "record-Gvb0Bg0Jfxfv0q8Fb2pXqKjg"
dataset <- glue("{projectid}:{rid}")
dataset

project-GvZyZ20J81vgPJGbJy8pgpyq:record-Gvb0Bg0Jfxfv0q8Fb2pXqKjg

In [4]:
setwd("/opt/notebooks")
cmd <- glue("dx extract_dataset {dataset} -ddd")
system(cmd)
dict_files <- list.files(pattern="codings|data_dictionary|entity_dictionary")
dict_files

[1] "app81499_20241105095754.dataset.codings.csv"          
[2] "app81499_20241105095754.dataset.data_dictionary.csv"  
[3] "app81499_20241105095754.dataset.entity_dictionary.csv"

In [11]:
data_dict = fread(dict_files[grepl("data_dictionary", dict_files)])

codes_to_extract = list(eid                = list(name="eid",       entity="participant", search="matches"),
                        sex                = list(name="p31",       entity="participant", search="matches"),
                        age                = list(name="p21022",    entity="participant", search="matches"),
                        ethnicity          = list(name="p21000",    entity="participant", search="matches"),
                        genetic_sex        = list(name="p22001",    entity="participant", search="matches"),
                        genetic_ethnicity  = list(name="p22006",    entity="participant", search="matches"),
                        pc1                = list(name="p22009_a1", entity="participant", search="matches"),
                        pc2                = list(name="p22009_a2", entity="participant", search="matches"),
                        pc3                = list(name="p22009_a3", entity="participant", search="matches"),
                        pc4                = list(name="p22009_a4", entity="participant", search="matches"),
                        pc5                = list(name="p22009_a5", entity="participant", search="matches"),
                        self_rep_ill       = list(name="p20002",    entity="participant", search="startswith"),
                        self_rep_ill_year  = list(name="p20008",    entity="participant", search="startswith"),
                        self_rep_proc      = list(name="p20004",    entity="participant", search="startswith"),
                        self_rep_proc_year = list(name="p20010",    entity="participant", search="startswith"))

filt_data_dict = lapply(codes_to_extract, function(x) {
    if (x$search=="matches") {
        data_dict[entity==x$entity & name==x$name]
    } else if (x$search=="startswith") {
        data_dict[entity==x$entity & grepl(paste0("^", x$name), name)]
    }
}) |> rbindlist(idcol = "item")

filt_data_dict[, field_id := paste(entity, name, sep=".")]
head(filt_data_dict, 3)

item,entity,name,type,primary_key_type,coding_name,concept,description,folder_path,is_multi_select,is_sparse_coding,linkout,longitudinal_axis_type,referenced_entity_field,relationship,title,units,field_id
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<lgl>,<lgl>,<chr>,<chr>,<chr>,<chr>,<lgl>,<chr>,<chr>,<chr>,<chr>,<chr>
eid,participant,eid,string,global,,NA,NA,Participant Information,,,,NA,,,Participant ID,,participant.eid
sex,participant,p31,integer,,data_coding_9,NA,NA,Population characteristics > Baseline characteristics,,,http://biobank.ctsu.ox.ac.uk/crystal/field.cgi?id=31,NA,,,Sex,,participant.p31
age,participant,p21022,integer,,,NA,NA,Population characteristics > Baseline characteristics,,,http://biobank.ctsu.ox.ac.uk/crystal/field.cgi?id=21022,NA,,,Age at recruitment,years,participant.p21022


In [8]:

#cmd <- glue("dx extract_dataset {dataset} --fields {fields} -o cohort_data.csv")
#system(cmd)

[1] "participant.eid,participant.p31"

In [ ]:
data_df <- fread("cohort_data.csv")
head(data_df)